# __Homework:__ Data talks club data engineering zoomcamp Data loading workshop
Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.
Here are the exercises we will do.

## 1. Use a generator
Remember the concept of generator? Let's practice using them to futher our understanding of how they work.
Let's define a generator and then run it as practice.

__Answer the following questions:__

_(I suggest practicing these questions without GPT as the purpose is to further your learning.)_


#### __Q1 - What is the sum of the outputs of the generator for limit = 5?__

In [1]:
import dlt
import duckdb

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum_ = 0.0

for sqrt_value in generator:
    sum_ += sqrt_value
    
print(sum_)

8.382332347441762




#### __Q2 - What is the 13th number yielded?__

In [2]:
limit = 13
generator = square_root_generator(limit)

i = 0
for sqrt_value in generator:
    i += 1
    if limit == i:
        print(f'{i}: {sqrt_value}')

13: 3.605551275463989



## 2. Append a generator to a table with existing data

Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data.

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.

In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
								table_name="people_01",
								write_disposition="replace")

info = generators_pipeline.run(people_2(),
								table_name="people_01",
								write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/peter/Sources/de-zoomcamp/_workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708378485.313148 is LOADED and contains no failed jobs


In [8]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

rides = conn.sql("SELECT * FROM people_01").df()
display(rides)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_01           │
│ people_02           │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708378519.994505,x4WUYtkh/rtStQ,None
1,2,Person_2,27,City_A,1708378519.994505,zuIHSbvgn8DUEA,None
2,3,Person_3,28,City_A,1708378519.994505,MeqgQXnxiap4Cw,None
3,4,Person_4,29,City_A,1708378519.994505,Fq1HgAh0FQEgww,None
4,5,Person_5,30,City_A,1708378519.994505,2R93B8rWQwzQdg,None
5,3,Person_3,33,City_B,1708378520.552543,PxG28QnGepqcTw,Job_3
6,4,Person_4,34,City_B,1708378520.552543,FBHCJfCGeKobIw,Job_4
7,5,Person_5,35,City_B,1708378520.552543,cCOtKjDYB1Wpkw,Job_5
8,6,Person_6,36,City_B,1708378520.552543,eG9QfRSPJcRnyA,Job_6
9,7,Person_7,37,City_B,1708378520.552543,ocJHCcL8ajl74Q,Job_7


#### __Q3.__ _(Append the 2 generators.)_ __Calculate the sum of all ages of people!__

In [12]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
								table_name="people_01",
								write_disposition="replace")

info = generators_pipeline.run(people_2(),
								table_name="people_01",
								write_disposition="append")

print(info)


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

people_01 = conn.sql("SELECT * FROM people_01").df()

sum_ages_01 = conn.sql("SELECT SUM(age) FROM people_01").df()
display(sum_ages_01)

Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/peter/Sources/de-zoomcamp/_workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708378631.5556102 is LOADED and contains no failed jobs


,sum(age)
0,353.0



## 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

#### __Q4.__ _(Merge the 2 generators using the ID column.)_ __Calculate the sum of ages of all the people!__

In [14]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
                    table_name="people_02",
                    write_disposition="replace")

print(info)


info = pipeline.run(people_2(),
                    table_name="people_02",
                    write_disposition="merge",
                    primary_key="id")

print(info)


conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

people_02 = conn.sql("SELECT * FROM people_02").df()

sum_ages_02 = conn.sql("SELECT SUM(age) FROM people_02").df()
display(sum_ages_02)

Pipeline dlt_ipykernel_launcher load step completed in 0.15 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/peter/Sources/de-zoomcamp/_workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708378668.423502 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/peter/Sources/de-zoomcamp/_workshop_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708378668.978562 is LOADED and contains no failed jobs


,sum(age)
0,266.0
